In [ ]:

from bs4 import BeautifulSoup
import requests

import api_keys # file containing token for musixmatch api
import csv
import pandas as pd

In [ ]:

def scrape_lyrics(url):
  lyric_response = requests.get(url, headers=HEADERS)
  soup = BeautifulSoup(lyric_response.text, "html.parser")

  content = soup.findAll(class_="lyrics__content__ok", text=True)
  if content:
    lyrics = "\n".join(x.text for x in content)
  elif soup.find(class_="lyrics__content__warning", text=True):
    data = soup.find(class_="lyrics__content__warning", text=True)
    lyrics = data.get_text()
  else:
    lyrics = None
  return lyrics


In [ ]:
KEY = api_keys.key
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

def get_song_lyrics(row):

  artist = row['artist']
  track = row['song_name']
  album = row['album']

  request_str = f' https://api.musixmatch.com/ws/1.1/matcher.track.get?apikey={KEY}&q_artist={artist}&q_track={track}&q_album={album}&f_has_lyrics=1'
  response = requests.get(request_str)

  if response.ok:
    # print(response.json()['message'])
    message_body = response.json()['message']['body']
    if 'track' in message_body:
      url_link = message_body['track']['track_share_url']
      lyrics = scrape_lyrics(url_link)
      row['lyrics'] = lyrics
  else:
    row['lyrics'] = None

  return row



#### Pop

In [ ]:
pop_df = pd.read_csv('pop_dataset.csv')

In [ ]:
pop_df.head()

,artist,album,song_name
0,Drake,Her Loss,I Guess It’s Fuck Me
1,Drake,Views,One Dance
2,The Weeknd,Starboy,Stargirl Interlude
3,Ed Sheeran,+,The A Team
4,Bryan Adams,Reckless (30th Anniversary / Deluxe Edition),Summer Of '69


In [ ]:
pop_lyrics_200 = pop_df[:200].apply(get_song_lyrics, axis=1)

In [ ]:
pd.set_option('display.max_colwidth', 0)

pop_lyrics_200.head()

,album,artist,lyrics,song_name
0,Her Loss,Drake,"You said f- me, and I was like, ""Cool""\nSo, now what′s the problem?\n\nDon't go hiding again\nTired of asking where you′ve been\nYou left so abruptly\nI guess it's f- me\nTell me, what did I do wrong?\nTell me, what did I do wrong?\nTell me, what did I do wrong?\nTell me, what did I do wrong?\n\nBefore it's over\nI need you to come over once again\nAnd before you give me closure\nNeed you to come a little bit closer\n\nStill steppin′ like Omega Psi Phi for mine\nIf bein′ real was a crime, I'd be doin′ life\nLike that n- not nice, I don't know polite\nI′m the first ever antisocial socialite\nThe pain that I seen in my mother's eyes in 2009\nHave me workin′ 'til it's 2049\nAin′t get hate when I tell you, ""Oh, some other time""\nLike I really got some other time\n\nYou just tell me, ""Never mind""\nKnow I sound crazy to a lazy mind\nKnow it′ll be a different kind\nShout out to the people workin' nine to five\nI be workin′ nine to nine\nAnd the six upside down, it's a nine\nYou already know the vibe\n\nAnd they leave ′cause of pride, but they comin' back every time\nThe devils that I recognize, most of them got pretty eyes\nT- and some plans of just gettin′ by, that's the way they live or die\nEasy to judge, but, girl, who am I?\n\nTruth or dare, I'ma take a double dare\nTruth is a suicide\nI would rather live a lie, keep you on the outside\nIntroduce you to the guys\nHave you throwin′ up the south side like you′re one of mine\nBut you're not one of mine\nYou belong to everybody else when you′re bored in your free time\n\nS- could make a thug cry, play it like a tough guy\nCouldn't even land in the Hamptons, they didn′t have the stairs for the s- I fly\nSwear it's like a metaphor, I can′t even get down from the s- I climb\nCan't even get down from the s- I climb\n\nYeah\nGirl, you're my size\nMake me tell you one time\nWe was on the front line, s- was in my bloodline\nWaitin′ for the sunshine\nBut the sun never shines on me, on me\n\nSleepin′ in the whip sometimes, girl, I was sleepin' upright\nHenny, red cup life, broski kept it tucked tight\nN- talkin′ bad s- about what they gon' do to mine\nAw, now their tongues tied\n\nWe was smokin′, watched the sun climb\nI would trap until my thumbs cried\nTryna change it all in one line\nNever seen a thug cry\nLove, it's been a long time\nBet you never seen a thug cry\nHit me on my hotline\nAnd no, ma, I′m not fine, at all\n\nDon't go hiding again\nTired of asking where you've been\nYou left so abruptly\nI guess it′s f- me\nTell me, what did I do wrong?\nTell me, what did I do wrong?\nTell me, what did I do wrong?\nTell me, what did I do wrong?\n\nBefore it′s over\nI need you to come over once again\nAnd before you give me closure\nNeed you to come a little bit closer",I Guess It’s Fuck Me
1,Views,Drake,"Baby, I like your style\n\nGrips on your waist, front way, back way\nYou know that I don′t play\nStreets not safe but I never run away\nEven when I'm away\nOti, oti\nThere′s never much love when we go OT\nI pray to make it back in one piece\nI pray, I pray\n\nThat's why I need a one dance\nGot a Hennessy in my hand\nOne more time 'fore I go\nHigher powers takin′ a hold on me\n\nI need a one dance\nGot a Hennessy in my hand\nOne more time ′fore I go\nHigher powers takin' a hold on me\n\nBaby, I like your style\n\nStrength and guidance\nAll that I′m wishing for my friends\nNobody makes it from my ends\nI had to bust up the silence\nYou know you gotta stick by me\nSoon as you see the text, reply me\nI don't wanna spend time fighting\nWe′ve got no time\n\nAnd that's why I need a one dance\nGot a Hennessy in my hand\nOne more time ′fore I go\nHigher powers taking a hold on me\n\nI need a one dance\nGot a Hennessy in my hand\nOne more time 'fore I go\nHigher powers taking a hold on me\n\nGot a pretty girl and she love me long time\nWine it, wine it, very long time\nOh, yeah, very long time\nBack up, back up, back up and wine it\nBack up, back u

In [ ]:
pop_lyrics_200.to_csv('pop_with_lyrics.csv',index=False)

#### Country

In [ ]:
country_df = pd.read_csv('hip_hop_dataset.csv')
country_lyrics_200 = country_df.apply(get_song_lyrics, axis=1)

In [ ]:
# find examples w no lyrics found
country_lyrics_200.loc[country_lyrics_200['lyrics'].isna(),:]

,album,artist,lyrics,song_name
3,Hip Hop Beats,Wale,NaN,Poke It Out
4,Greatest Hits,Fugees,None,Vocab - Refugees Hip Hop Remix
5,Dance Hip-Hop,ICE-T,None,Lifestyles of the Rich and Infamous
9,Türkishe Hip Hop & Rap 2022,Gazapizm,NaN,İsminizi İstiyorlar - Live In İzmir / 2019
13,Go! Common Classics,Erykah Badu,None,Love Of My Life (An Ode To Hip Hop) - Longer A...
...,...,...,...,...
589,Cheiro do Mato (Acústico),Hungria Hip Hop,None,Amor e Fé - Acústico
590,Summer Hip Hop & RnB,City High,None,What Would You Do?
592,Ain't A Damn Thing Changed,Nice & Smooth,None,Hip Hop Junkies
593,90s Underground Hip Hop,Crooklyn Dodgers 95',None,Return of The Crooklyn Dodgers


In [ ]:
# run if non-NaNs found 

country_lyrics_200.to_csv('country_with_lyrics.csv',index=False)